# SQL Murder Mystery

## Can you find out whodunnit?

![A decorative illustration of a detective looking at an evidence board.](https://mystery.knightlab.com/174092-clue-illustration.png)

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

## SQL sleuths start here

A crime has taken place and the detective needs your help. The detective gave you the crime scene report, but you somehow lost it. You vaguely remember that the crime was a **​murder​**that occurred sometime on ​**Jan.15, 2018​** and that it took place in ​**SQL City​**. Start by retrieving the corresponding crime scene report from the police department’s database.

### Exploring the Database Structure

Experienced SQL users can often use database queries to infer the structure of a database. But each database system has different ways of managing this information. The SQL Murder Mystery is built using SQLite. Use this SQL command to find the tables in the Murder Mystery database.

Run this query to find the names of the tables in this database.

This command is specific to SQLite. For other databases, you'll have to learn their specific syntax.


In [12]:
# Install required packages
%pip install jupysql sqlalchemy pandas --quiet

# Load SQL magic
%load_ext sql

# Connect to the database
%sql sqlite:///sql-murder-mystery.db
#%config SqlMagic.style = 'table'

Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [13]:
%%sql
SELECT name
FROM sqlite_master

Running query in 'sqlite:///sql-murder-mystery.db'

name
crime_scene_report
drivers_license
facebook_event_checkin
interview
get_fit_now_member
sqlite_autoindex_get_fit_now_member_1
get_fit_now_check_in
solution
check_solution
income



Besides knowing the table names, you need to know how each table is structured. The way this works is also dependent upon which database technology you use. Here's how you do it with SQLite.

Run this query to find the structure of the `crime_scene_report` table

Change the value of 'name' to see the structure of the other tables you learned about with the previous query.


In [14]:
%%sql
SELECT sql
FROM sqlite_master
where name = 'crime_scene_report'

Running query in 'sqlite:///sql-murder-mystery.db'

sql
"CREATE TABLE crime_scene_report ( date integer, type text, description text, city text )"


In [15]:
%%sql
SELECT *
FROM crime_scene_report
WHERE type = "murder" AND date = "20180115" AND city = "SQL City"

Running query in 'sqlite:///sql-murder-mystery.db'

date,type,description,city
20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [16]:
%%sql
SELECT *
FROM person
WHERE address_street_name = "Northwestern Dr"
ORDER BY address_number DESC
LIMIT 1
# first witness is morty schapiro (ID 14887)

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949


In [17]:
%%sql
SELECT *
FROM person
WHERE name LIKE '%Annabel%' AND address_street_name = "Franklin Ave"
# second person is Annabel Miller (ID 16371)

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn
16371,Annabel Miller,490173,103,Franklin Ave,318771143


In [18]:
%%sql
SELECT *
FROM interview
WHERE person_id = "16371"
# Annabels interview

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [19]:
%%sql
SELECT *
FROM interview
WHERE person_id = "14887"
# Mortys interview

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [20]:
%%sql
SELECT *
FROM get_fit_now_member
WHERE membership_status = "gold" AND id LIKE '%48Z%'
# found 2 suspects!

Running query in 'sqlite:///sql-murder-mystery.db'

id,person_id,name,membership_start_date,membership_status
48Z7A,28819,Joe Germuska,20160305,gold
48Z55,67318,Jeremy Bowers,20160101,gold


In [21]:
%%sql
SELECT *
FROM person p
JOIN drivers_license d
  ON p.license_id = d.id
WHERE p.id IN (28819, 67318)
  AND d.plate_number LIKE '%H42W%';
# found the murderer!!! ID = 67318


Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn,id_1,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS



### The rest is up to you!

If you're really comfortable with SQL, you can probably get it from here. To help, here is the schema diagram:

![schema diagram](schema.png)

Use your knowledge of the database schema and SQL commands to find out who committed the murder.
### Check your solution

Did you find the killer? When you think you know the answer, submit your suspect using the following code and find out if you're right.


In [ ]:
%%sql
INSERT INTO solution VALUES (1, 'Jeremy Bowers');
SELECT value FROM solution;

# It was fine before but something happened and it is now experiencing an error

# RuntimeError: (sqlite3.OperationalError) database is locked
# [SQL: INSERT INTO solution VALUES (1, 'Jeremy Bowers');]
# (Background on this error at: https://sqlalche.me/e/20/e3q8)

Running query in 'sqlite:///sql-murder-mystery.db'

RuntimeError: (sqlite3.OperationalError) database is locked
[SQL: INSERT INTO solution VALUES (1, 'Jeremy Bowers');]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [23]:
%%sql
SELECT *
FROM interview
WHERE person_id = "67318"

Running query in 'sqlite:///sql-murder-mystery.db'

person_id,transcript
67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017."


In [ ]:
%%sql
SELECT *
FROM person p
JOIN drivers_license d
    ON p.license_id = d.id
WHERE (d.car_make = "Tesla" AND d.car_model = "Model S")
    AND (d.height = "65" OR d.height = "67")
    AND d.hair_color = "red"
    AND d.gender = "female"

# RED KORB!!!

Running query in 'sqlite:///sql-murder-mystery.db'

id,name,license_id,address_number,address_street_name,ssn,id_1,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
78881,Red Korb,918773,107,Camerata Dr,961388910,918773,48,65,black,red,female,917UU3,Tesla,Model S
